# Data Gathering and Data Cleaning
#### By: _Zhan Yu_

## Table of Contents
- [Loading Libraries & Data](#Loading-Libraries-&-Data)
    -[Socrata](#Socrata)
    -[Census Data](#Census-Data)
- [Data Preprocessing](#Data-Preprocessing)
    - [rats.csv](#rats.csv)
    - [restaurants.csv](#restaurants.csv)
    - [rat_sightings.csv](#rat_sightings.csv)
- [Source](#Source)

## Loading Libraries & Data

In this project, the datasets [Rodent Inspection in NYC](https://data.cityofnewyork.us/Health/Rodent-Inspection/p937-wjvj), [Rat Sightings](https://data.cityofnewyork.us/Social-Services/Rat-Sightings/3q43-55fe) and [Restaurant-Inspection](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j ) are public datasets from data.cityofnewyork.us. The census data are from [United States Census Bureau](https://www.census.gov/en.html).

In [1]:
# Libraries: 
import pandas as pd
import numpy as np
import os
import requests
import json

from sodapy import Socrata

import warnings
warnings.simplefilter(action="ignore")

### Socrata 

The Socrata APIs provide rich query functionality through a query language we call the “Socrata Query Language” or “SoQL”.   
Install packages at the current environment (for example, mine is (dsi)) before running:
``` Terminal
pip install sodapy
```
From [NYC Open Data](https://opendata.cityofnewyork.us/), we can load Rodent Inspection dataset, NYC Restaurant Inspection dataset and Rat Sightings dataset by using `Socrata`.   
Unauthenticated client only works with public data sets. Note `None` in place of application token, and no username or password.   
In these three cases they are all public datasets.

In [2]:
client = Socrata("data.cityofnewyork.us", None)

# First 1,000,000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("p937-wjvj",                                # Rodent Inspection dataset
                     limit=1_000_000, where="boro_code = 1")

# Convert to pandas DataFrame
rats = pd.DataFrame.from_records(results)

In [4]:
# First 150,000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.

results = client.get("3q43-55fe",                                # Rat Sightings dataset
                     limit=150_000)

# Convert to pandas DataFrame
rat_sightings = pd.DataFrame.from_records(results)

Because the datasets are too big (about 200 MB), we are going to do some data cleaning and trimming before we export them.

### Restaurant Data

From [United States Census Bureau](https://www.census.gov/en.html) we will use "ZIP Codes Business Patterns" from 2010 to 2017. The API key is not needed.

In [67]:
zip_manhattan = set(rats['zip_code'].dropna().astype(int))

# Remove unwanted zip codes:
zip_manhattan.remove(0)
zip_manhattan.remove(10000) 

In [104]:
baseAPI = f"https://api.census.gov/data/2012/zbp?get=ESTAB&for=zipcode:10002&NAICS2012=722"
response = requests.get(baseAPI)

formattedResponse = json.loads(response.text)[1:]
formattedResponse = [item[::-1] for item in formattedResponse]

In [105]:
formattedResponse

[['10002', '722', '610']]

In [107]:
# 2010 - 2017 Business Patterns:
zip_res_yr = pd.DataFrame(columns=['zipcode', 'type', 'count', 'year'], data=[]) 

# From 2010 to 2011:
for year in range(2010,2012):
    
    #zipcode in Manhattan:
    for zipcode in zip_manhattan:
        
        
        try:
            # "ESTAB": Number of establishments
            # "72": "Food services and drinking places"
            baseAPI = f"https://api.census.gov/data/{year}/zbp?get=ESTAB&for=zipcode:{zipcode}&NAICS2007=72"
            response = requests.get(baseAPI)
            formattedResponse = json.loads(response.text)[1:]
            formattedResponse = [item[::-1] for item in formattedResponse]
            
            # Store the response in a dataframe
            zip_res = pd.DataFrame(columns=['zipcode','type', 'count'], data=formattedResponse)
            zip_res['year'] = year
            zip_res_yr = pd.concat([zip_res_yr, zip_res], ignore_index=True)
            
        except:
            pass

# From 2012 to 2016:
for year in range(2012,2017):
    
    #zipcode in Manhattan:
    for zipcode in zip_manhattan:
        
        
        try:            
            # "ESTAB": Number of establishments
            # "722": "Food services and drinking places"
            baseAPI = f"https://api.census.gov/data/{year}/zbp?get=ESTAB&for=zipcode:{zipcode}&NAICS2012=722"
            response = requests.get(baseAPI)
            formattedResponse = json.loads(response.text)[1:]
            formattedResponse = [item[::-1] for item in formattedResponse]
            
            # Store the response in a dataframe
            zip_res = pd.DataFrame(columns=['zipcode','type', 'count'], data=formattedResponse)
            zip_res['year'] = year
            zip_res_yr = pd.concat([zip_res_yr, zip_res], ignore_index=True)
            
        except:
            pass

# Year 2017        
for zipcode in zip_manhattan:
    try:            
        # "ESTAB": Number of establishments
        # "722": "Food services and drinking places"
        baseAPI = f"https://api.census.gov/data/2017/zbp?get=ESTAB&for=zipcode:{zipcode}&NAICS2017=722"
        response = requests.get(baseAPI)
        formattedResponse = json.loads(response.text)[1:]
        formattedResponse = [item[::-1] for item in formattedResponse]
            
        # Store the response in a dataframe
        zip_res = pd.DataFrame(columns=['zipcode','type', 'count'], data=formattedResponse)
        zip_res['year'] = 2017
        zip_res_yr = pd.concat([zip_res_yr, zip_res], ignore_index=True)
            
    except:
        pass
zip_res_yr= zip_res_yr.astype(int)


In [111]:
zip_res_yr['zip_year'] = zip_res_yr['zipcode'].astype(str) + ' ' + zip_res_yr['year'].astype(str)
zip_res_yr['zip_year_month'] = ''
zip_res_yr.head()

,zipcode,type,count,year,zip_year,zip_year_month
0,10271,72,2,2010,10271 2010,
1,10278,72,1,2010,10278 2010,
2,10279,72,1,2010,10279 2010,
3,10280,72,9,2010,10280 2010,
4,10281,72,15,2010,10281 2010,


In [112]:
zip_res_yr = zip_res_yr[['zip_year', 'count']]
zip_res_yr.head()

,zip_year,count
0,10271 2010,2
1,10278 2010,1
2,10279 2010,1
3,10280 2010,9
4,10281 2010,15


In [113]:
# Export data as name "zip_res_yr.csv":
zip_res_yr.to_csv('../datasets/zip_res_yr.csv', index=False)

### Census Data

From [United States Census Bureau](https://www.census.gov/en.html) we will use "ACS 5-Year Data" from 2011 to 2018 and "Decennial Census" of 2010.  
First we need to get our an API key from [HERE](https://api.census.gov/data/key_signup.html).

In [5]:
api_key = '9991b4ab66903673ef2f2f5dae3ec63d2ab4f4f2'

In [6]:
# Initiate an empty data frame 'zip_pop_yr' which has columns 'zipcode', 'population' and 'year':
zip_pop_yr = pd.DataFrame(columns=['zipcode', 'population', 'year'], data=[]) 
# 2010 Census data:
for zipcode in zip_manhattan:
    try:
        baseAPI = f"https://api.census.gov/data/2010/dec/sf1?key={api_key}&get=H010001&for=zip%20code%20tabulation%20area:{zipcode}"
        response = requests.get(baseAPI)
        formattedResponse = json.loads(response.text)[1:]
        formattedResponse = [item[::-1] for item in formattedResponse]
        
        # Store the response in a dataframe
        zip_pop = pd.DataFrame(columns=['zipcode', 'population'], data=formattedResponse)
        zip_pop['year'] = 2010
        zip_pop_yr = pd.concat([zip_pop_yr, zip_pop], ignore_index=True)
        
    except:
        pass

In [7]:
# 2011-2018 American Community Survey 5-Year Data:
acs_table = 'B01003_001E'  # Code of "Total population" 

# From 2011 to 2018:
for year in range(2011,2019):
    
    #zipcode in Manhattan:
    for zipcode in zip_manhattan:
        
        try:
            
            baseAPI = f"https://api.census.gov/data/{year}/acs/acs5?key={api_key}&get={acs_table}&for=zip%20code%20tabulation%20area:{zipcode}"
            response = requests.get(baseAPI)
            formattedResponse = json.loads(response.text)[1:]
            formattedResponse = [item[::-1] for item in formattedResponse]
            
            # Store the response in a dataframe
            zip_pop = pd.DataFrame(columns=['zipcode', 'population'], data=formattedResponse)
            zip_pop['year'] = year
            zip_pop_yr = pd.concat([zip_pop_yr, zip_pop], ignore_index=True)
            
        except:
            pass
        
zip_pop_yr= zip_pop_yr.astype(int)


In [8]:
zip_pop_yr.dtypes

zipcode       int64
population    int64
year          int64
dtype: object

In [9]:
zip_pop_yr.to_csv('../datasets/zip_pop_yr.csv', index=False)

## Data Preprocessing

### rats.csv  

In [10]:
# Setting the index to 'job_id':
rats.set_index('job_id',inplace = True)
rats.head()

,inspection_type,job_ticket_or_work_order_id,job_progress,bbl,boro_code,block,lot,house_number,street_name,zip_code,x_coord,y_coord,latitude,longitude,borough,inspection_date,result,approved_date,location
job_id,,,,,,,,,,,,,,,,,,,
PO2075886,INITIAL,2075886,1,1022430001,1,02243,0001,4977,BROADWAY,10034,1006543,255608,40.868203550611,-73.919469362846,Manhattan,2019-08-02T11:11:55.000,Active Rat Signs,2019-08-07T11:34:09.000,"{'latitude': '40.868203550611', 'longitude': '..."
PO333717,BAIT,70581,5,1009620100,1,00962,0100,462,1 AVENUE,10016,991147,208600,40.739489790595,-73.976623458224,Manhattan,2011-10-26T09:45:58.000,Bait applied,2011-10-28T07:14:07.000,"{'latitude': '40.739489790595', 'longitude': '..."
PO431003,BAIT,92841,1,1022290001,1,02229,0001,4986,BROADWAY,10034,1006611,255630,40.868266491555,-73.919201730342,Manhattan,2012-03-19T14:00:03.000,Bait applied,2012-03-20T07:20:09.000,"{'latitude': '40.868266491555', 'longitude': '..."
PO333717,BAIT,69386,4,1009620100,1,00962,0100,462,1 AVENUE,10016,991147,208600,40.739489790595,-73.976623458224,Manhattan,2011-10-17T13:50:37.000,Bait applied,2011-10-18T07:09:09.000,"{'latitude': '40.739489790595', 'longitude': '..."
PO422674,BAIT,91531,1,1022290001,1,02229,0001,4986,BROADWAY,10034,1006611,255630,40.868266491555,-73.919201730342,Manhattan,2012-03-08T13:51:50.000,Bait applied,2012-03-09T12:23:58.000,"{'latitude': '40.868266491555', 'longitude': '..."


In [11]:
# Check the shape of data frame:
rats.shape

(594536, 19)

After taking a first look at our dataset, we have a general idea of features of dataset. We are going to only keep the features we need.

In [12]:
rats_df = rats[['inspection_type',
                'zip_code','inspection_date']]
rats_df.head()

,inspection_type,zip_code,inspection_date
job_id,,,
PO2075886,INITIAL,10034,2019-08-02T11:11:55.000
PO333717,BAIT,10016,2011-10-26T09:45:58.000
PO431003,BAIT,10034,2012-03-19T14:00:03.000
PO333717,BAIT,10016,2011-10-17T13:50:37.000
PO422674,BAIT,10034,2012-03-08T13:51:50.000


In [13]:
rats_df.isnull().mean()

inspection_type    0.000000
zip_code           0.004043
inspection_date    0.000000
dtype: float64

Since the missing values have very small percentage of total dataset, we can drop all the rows with missing values and still have a relatively large dataset.

In [14]:
rats_df.dropna(inplace = True)
rats_df.shape

(592132, 3)

In [15]:
rats_df.isnull().sum().sum()

0

We are going to check and change the original data types:

In [16]:
# Checking the original data types:
rats_df.dtypes

inspection_type    object
zip_code           object
inspection_date    object
dtype: object

In [17]:
# Changing date columns into datetime form: 
rats_df['inspection_date'] = pd.to_datetime(rats_df['inspection_date'])

# Changing those columns which seem like numerical but actually strings:
rats_df['zip_code']=rats_df['zip_code'].astype(int)

# Checking data types again:
rats_df.dtypes

inspection_type            object
zip_code                    int64
inspection_date    datetime64[ns]
dtype: object

We are setting out dataset in chronological order because the time range of our data set is more than 100 years.

In [18]:
rats_df = rats_df.sort_values(by = 'inspection_date', ascending = False)
rats_df.head()

,inspection_type,zip_code,inspection_date
job_id,,,
PO2167036,COMPLIANCE,10029,2020-01-08 13:20:54
PO2167008,COMPLIANCE,10029,2020-01-08 13:10:44
PO2189724,INITIAL,10002,2020-01-08 13:10:40
PO2125410,COMPLIANCE,10019,2020-01-08 13:05:13
PO2167016,COMPLIANCE,10029,2020-01-08 13:00:09


Let's only see the last 10 years data which are still the majority of data:

In [37]:
rats_df = rats_df.loc[(rats_df['inspection_date'] >= '2010-01-01')&(rats_df['inspection_date'] < '2019-01-01')]
rats_df = rats_df.loc[(rats_df['zip_code']>10000)&(rats_df['zip_code']<20000)]
rats_df.shape

(510172, 3)

In [38]:
# Export the cleaned data:
rats_df.to_csv('../datasets/rats.csv', index=False)

### restaurants.csv

In [21]:
restaurants.shape

(401404, 26)

In [22]:
restaurants = restaurants.loc[restaurants['boro'] == 'Manhattan']

# Setting the index to 'job_id':
restaurants.set_index('camis',inplace = True)
restaurants.head(2)

,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,violation_code,...,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,grade,grade_date
camis,,,,,,,,,,,,,,,,,,,,,
40610087,RUSSIAN VODKA ROOM,Manhattan,265,WEST 52 STREET,10019,2123075835,Russian,2019-03-30T00:00:00.000,Violations were cited in the following area(s).,10F,...,40.763175639217,-73.984315014892,105,03,013100,1024825,1010240001,MN17,NaN,NaN
41046726,THE CAPITAL GRILLE,Manhattan,155,EAST 42 STREET,10017,2129532000,Steak,2018-04-12T00:00:00.000,Violations were cited in the following area(s).,06E,...,40.751173988368,-73.975406637538,106,04,009200,1036160,1012970031,MN19,A,2018-04-12T00:00:00.000


In [23]:
restaurants_df = restaurants[['inspection_date','zipcode']]
restaurants_df.head()

,inspection_date,zipcode
camis,,
40610087,2019-03-30T00:00:00.000,10019
41046726,2018-04-12T00:00:00.000,10017
41540529,2016-09-08T00:00:00.000,10003
50072484,2020-01-07T00:00:00.000,10128
50092747,2019-06-27T00:00:00.000,10010


In [24]:
restaurants_df.isnull().mean()

inspection_date    0.000000
zipcode            0.016684
dtype: float64

In [25]:
# Changing date columns into datetime form: 
restaurants_df['inspection_date'] = pd.to_datetime(restaurants_df['inspection_date'])

In [26]:
# We are going to keep the data from 2010 to 2018:
restaurants_df = restaurants_df.loc[(restaurants_df['inspection_date'] >= '2010-01-01')&(restaurants_df['inspection_date'] < '2019-01-01')]
restaurants_df.shape

(88308, 2)

In [72]:
restaurants_df.dropna(inplace = True)
restaurants_df['zipcode'] = restaurants_df['zipcode'].astype(int)
restaurants_df = restaurants_df.loc[restaurants_df['zipcode']>10000]
restaurants_df.shape

(86673, 2)

In [73]:
# Export data as name "rat_sightings.csv":
restaurants_df.to_csv('../datasets/restaurants_df.csv', index=False)

### rat_sightings.csv

In [29]:
rat_sightings.shape

(142889, 36)

In [30]:
rat_sightings.head(2)

,descriptor,incident_zip,x_coordinate_state_plane_,created_date,location,city,:@computed_region_sbqj_enih,cross_street_2,:@computed_region_efsh_h5xi,park_facility_name,...,longitude,:@computed_region_f5dn_yrer,status,unique_key,intersection_street_2,closed_date,resolution_action_updated_date,address_type,due_date,facility_type
0,Rat Sighting,10023,987231,2020-03-01T00:32:41.000,"{'latitude': '40.77421945878715', 'human_addre...",NEW YORK,12,FREEDOM PLACE SOUTH,10091,Unspecified,...,-73.9892371845004,20,In Progress,45723202,FREEDOM PLACE SOUTH,NaN,NaN,NaN,NaN,NaN
1,Rat Sighting,11378,1013601,2020-02-29T23:00:22.000,"{'latitude': '40.72403295808778', 'human_addre...",MASPETH,62,69 PLACE,14788,Unspecified,...,-73.89410882860271,54,In Progress,45722503,69 PLACE,NaN,NaN,NaN,NaN,NaN
2,Rat Sighting,11103,1009282,2020-02-29T22:19:31.000,"{'latitude': '40.76639220669787', 'human_addre...",ASTORIA,72,28 AVENUE,16860,Unspecified,...,-73.90963328356786,39,In Progress,45725479,28 AVENUE,NaN,NaN,NaN,NaN,NaN
3,Rat Sighting,11374,1023190,2020-02-29T22:07:07.000,"{'latitude': '40.72997950637147', 'human_addre...",REGO PARK,70,98 STREET,14785,Unspecified,...,-73.85950151820727,40,In Progress,45722505,98 STREET,NaN,NaN,NaN,NaN,NaN
4,Rat Sighting,10472,1017766,2020-02-29T21:11:35.000,"{'latitude': '40.8274342564627', 'human_addres...",BRONX,26,WESTCHESTER AVENUE,11610,Unspecified,...,-73.87889462284318,58,In Progress,45724727,WESTCHESTER AVENUE,NaN,NaN,NaN,NaN,NaN


In [31]:
sightings = rat_sightings[['incident_zip', 'created_date', 'borough']]
sightings.head()

,incident_zip,created_date,borough
0,10023,2020-03-01T00:32:41.000,MANHATTAN
1,11378,2020-02-29T23:00:22.000,QUEENS
2,11103,2020-02-29T22:19:31.000,QUEENS
3,11374,2020-02-29T22:07:07.000,QUEENS
4,10472,2020-02-29T21:11:35.000,BRONX


In [32]:
sightings = sightings.loc[sightings['borough']=='MANHATTAN'].drop(columns = 'borough')
sightings = sightings.dropna()

In [33]:
sightings = sightings.drop(sightings.loc[sightings['incident_zip']=='N/A'].index)
sightings['incident_zip'] = sightings['incident_zip'].astype(int)

In [69]:
sightings = sightings.loc[(sightings['incident_zip']>10000)&(sightings['incident_zip']<20000)]
sightings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37042 entries, 0 to 142886
Data columns (total 2 columns):
incident_zip    37042 non-null int64
created_date    37042 non-null object
dtypes: int64(1), object(1)
memory usage: 868.2+ KB
